# Job launcher AWS

The following code was employed to launch an hybrid job simulating 3 different spectral gap for an Ising 1D chain of 4 sites, on the IonQ Aria 1 device available via AWS Braket.

It is basically what has been describe in `spectral_Ising_1D_ionq.ipynb` wrapper inside a `@hybrid_job` instance.

In [22]:
from braket.devices import Devices
from braket.aws import AwsDevice
from braket.error_mitigation import Debias
from braket.jobs.metrics import log_metric
from braket.jobs import InstanceConfig,hybrid_job

# choose a large instance type for our experiment
large_instance = InstanceConfig(instanceType="ml.m5.large")

#device = AwsDevice(Devices.IonQ.Aria1.value)

# print(Devices.IonQ.Aria1.value)


@hybrid_job(device=Devices.IonQ.Aria1, instance_config=large_instance)
def loop():
    import pennylane as qml
    import pennylane.numpy as np

    import braket.pennylane_plugin as bket
    
    tau_steps = 15#thermalization steps
    t_steps = 25 # time evolution steps
    
    JZ = 0 
    JX = 1 # only X coupling
    JY = 0
    
    n_qubits = 4
    hZ_s = np.linspace(2.2,14,15)
    d = {f'{hZ_s[5]}':[hZ_s[5], .25,.09],
         f'{hZ_s[6]}':[hZ_s[6], .25,.06],
         f'{hZ_s[7]}':[hZ_s[7], .25,.06]}
    
    
    
    ################################################
    dev_remote = qml.device(
                "braket.aws.qubit",
                device_arn=Devices.IonQ.Aria1.value,
                wires=4,
                shots=2500,
                device_parameters={"errorMitigation": Debias()}
                )
    #device = qml.device("lightning.qubit", wires=n_qubits,shots=2500)#"braket.local.qubit"
    @qml.qnode(dev_remote)#device
    def evol_SGS_expval(hX, JX=0, JY=0, JZ=0,tau_steps=10,t_steps=10, step=0, delta_tau = 0.3,delta_t = 0.3, O=qml.PauliX(0)):
        '''Circuit returning the expectation value of time evoluted X_0 on the SGS state obtained via Adiabatic Thermalization
        '''
        
            #######################################################
        def trotter_step(s,hZ, JX=0, JY=0, JZ=0, delta_t = 0.3):
            '''Trotter step function
            Params:
            s (float): performs  time evolution with tunable parameter:  s*hZ
            '''

            #X_nX_n+1 coupling
            if JX != 0:

                for i in range(0,n_qubits,2):
                    qml.IsingXX(-2*JX*delta_t,wires=[(i+1)%n_qubits, i])

                for i in range(1,n_qubits,2):
                    qml.IsingXX(-2*JX*delta_t,wires=[(i+1)%n_qubits, i])

                qml.Barrier(wires=range(n_qubits), only_visual=True)


            #Z term transverse field
            if hZ !=0 :
                for i in range(0,n_qubits):
                    qml.RZ(-s*delta_t*hZ,wires=i)
                qml.Barrier(wires=range(n_qubits), only_visual=True)


        def SGS_circ(hZ, JX=0, JY=0, JZ=0,tau_steps=10, delta_tau = 0.3):
            '''Circuit responsible for the preparation of the SGS state via Adiabatic Thermalization
            '''
            thermalization_steps = np.linspace(0, 1, tau_steps+2)[1:-1]#first and last does not affect
            #state preparation
            #########
            # |+++++..> is a good starting point for thermalization 
            for i in range(n_qubits):
                #bket.GPi2(np.pi/2, wires=i)
                qml.Hadamard(wires=i)
            for s in thermalization_steps:
                trotter_step(s,hZ,JX,JY,JZ,delta_tau)


        #### CHEBYSHEV TIMESTEPS
        def chebyshev_time(t_steps, delta_t):
            '''Using Chebyshev nodes as time to have better fit'''

            n_nodes = []

            for t in range(t_steps):
                n_nodes.append(t_steps*delta_t*(0.5*np.cos(np.pi*(t_steps-1-t)/(t_steps-1))+0.5))

            return n_nodes

        def time_evolution(hX, JX=0, JY=0, JZ=0,t_steps=10, step=0, delta_t = 0.3):
            times = chebyshev_time(t_steps, delta_t)

            for i,t in enumerate(times[:step]):#range(t_steps):
                if i==0:
                    dt=t
                else:
                    dt = t - times[i-1]
                trotter_step(1,hX,JX,JY,JZ, dt)
                
                
        SGS_circ(hX, JX, JY, JZ,tau_steps, delta_tau)

        time_evolution(hX, JX, JY, JZ,t_steps,step, delta_t)

        return qml.sample(op = O, )#qml.expval(O)

    shot_price = 0.03
    task_price = 0.3
    max_price = 2300
    circ_price = task_price+2500*shot_price
    
    
    bill = 0
    counter = 0 
    
    all_vals = {}
    all_errors = {}
    for k,v in d.items():
        hZ = v[0]
        delta_tau = v[1]
        delta_t = v[2]

        print(f'\n-----{hZ}-------')
        if bill+ circ_price > max_price:
                #print('We pay:',bill, 'max price',max_price)
                break
        vals = []
        errors = []
        tmp = list(range(0,t_steps,2))
        for t in tmp[:-3]:
            expval = 0
            samples = evol_SGS_expval(hZ, JX, JY, JZ,tau_steps=tau_steps,t_steps=t_steps,step=t,delta_t=delta_t,delta_tau=delta_tau)
            counts = {}
            counts[1] = np.sum(np.where(samples==1,1, 0 ))
            counts[-1] = np.sum(np.where(samples==-1,1, 0 ))

            shots = counts[1]+counts[-1]
            p_0 = counts[1]/shots
            p_1 = counts[-1]/shots

            expval = (p_1*(-1)+ p_0)
            err = np.sqrt(4*p_0*p_1/(shots-1))# /
            errors.append(err)
            vals.append(expval)
            counter += 1
            #log_metric(metric_name="expval", iteration_number=counter, value=expval)

            #if t%5==0:
            #print(t, f'{expval:.3f}, {err:.3f}')
            bill += circ_price#tracker.totals["executions"]*task_price+tracker.totals["shots"]*shot_price
            #print('we pay',bill)
            if bill+ circ_price > max_price:
                #print('We pay:',bill, 'max price',max_price)
                break


        all_vals[hZ] = vals
        all_errors[hZ] = errors
    
    #print(counter)
    return [all_vals, all_errors]
    

In [23]:
import time
from braket.aws import AwsQuantumJob

job = loop()
start = time.time()
while job.state() not in AwsQuantumJob.TERMINAL_STATES:
    print(time.time()-start,job.state())
    time.sleep(10)
print(job.state())
print(job.result())    
    
all_vals, all_errors = job.result()['result']
np.save('./all_vals.npy', all_vals)
np.save('./all_errors.npy', all_errors)




0.35386133193969727 QUEUED
10.593737602233887 RUNNING
20.805784702301025 RUNNING
31.02843403816223 RUNNING
41.270657539367676 RUNNING
51.4939661026001 RUNNING
61.71318030357361 RUNNING
71.9451379776001 RUNNING
82.15104293823242 RUNNING
92.38719439506531 RUNNING
102.63732552528381 RUNNING
112.85401177406311 RUNNING
123.09439015388489 RUNNING
133.35239791870117 RUNNING
143.5904312133789 RUNNING
153.84455370903015 RUNNING
164.07049226760864 RUNNING
174.28518176078796 RUNNING
184.50842690467834 RUNNING
194.73186826705933 RUNNING
204.96612811088562 RUNNING
215.18742990493774 RUNNING
225.3968358039856 RUNNING
235.62707424163818 RUNNING
245.85185384750366 RUNNING
256.06879711151123 RUNNING
266.3403148651123 RUNNING
276.54303336143494 RUNNING
286.7724847793579 RUNNING
296.9958415031433 RUNNING
307.4135642051697 RUNNING
317.66566157341003 RUNNING
327.9228045940399 RUNNING
338.14949655532837 RUNNING
348.3753390312195 RUNNING
358.6250545978546 RUNNING
368.89159965515137 RUNNING
379.14110016822815